
# 📚 Full-Corpus Chapter/Page Overlap & Correlation — Turnkey Notebook

This notebook processes *thousands* of Markdown files organized as:

```
main_folder/
  physics_chapters/   *.md
  physics_pages/      *.md
  chemistry_chapters/ *.md
  chemistry_pages/    *.md
  biology_chapters/   *.md
  biology_pages/      *.md
```

**It will:**
- Lowercase text and tokenize with **spaCy** (with/without stopwords)
- Compute per-document stats (tokens/types/TTR)
- Build **Jaccard overlap**, **TF‑IDF cosine**, and **Pearson correlation** matrices
- Save **CSVs + PNG heatmaps** and generate **index.html** with links

> ⚠️ **Tip:** Start with `chapters` only before you run `pages` on very large books.


In [5]:

# ⬇️ Install dependencies (run once per environment)
# If running locally, you may prefer `pip install -r requirements.txt` instead.
try:
    import spacy, pandas, numpy, sklearn, jinja2, matplotlib
except Exception as e:
    %pip install -q spacy==3.7.4 pandas>=2.0 numpy>=1.25 scikit-learn>=1.4 tqdm>=4.66 matplotlib>=3.8 jinja2>=3.1
    import spacy, pandas, numpy, sklearn, jinja2, matplotlib

# Download spaCy English model if missing
import spacy
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")
print("spaCy model ready.")

spaCy model ready.


In [ ]:

# ⚙️ Configuration — set your paths & options here

from pathlib import Path

ROOT = Path(".")                    # <- current directory
OUT  = Path("analysis_out")         # outputs (CSVs, PNGs, index.html)

LEVELS = ["chapters"]               # choose subset: ["chapters"] first, then pages
MODES  = ["intrabook", "interbook"] # what to compute
WITH_STOPWORDS = "both"             # "yes" | "no" | "both"
NGRAM_RANGE = (1, 2)                # TF-IDF n-grams
MIN_DF = 2                          # min doc freq for TF-IDF features
TOPK = 50                           # top tokens per (book, level)

# Compute Pearson correlation (dense step). Set to False to skip on huge corpora.
DO_PEARSON = False                 # Disable for memory optimization

# Safety: create output dirs
(OUT / "csv").mkdir(parents=True, exist_ok=True)
(OUT / "png").mkdir(parents=True, exist_ok=True)

print("Config set. Root =", ROOT.resolve())


Config set. Root = /Users/tanmayagarwal/Desktop/My_Computer/Columbia/Fall_2025/RA_LLM/main_folder


In [ ]:

import os, re, json, math, itertools
from collections import Counter, defaultdict
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from jinja2 import Template

MD_PATTERN = re.compile(r"[#>\*\_\-\[\]\(\)\!\`\~]")

def read_markdown_lower(path: Path) -> str:
    text = path.read_text(encoding="utf-8", errors="ignore").lower()
    return MD_PATTERN.sub(" ", text)

def tokenize_doc(nlp, text, remove_stop=False, lemmatize=True):
    # lightweight pipeline
    with nlp.select_pipes(enable=["tok2vec","tagger","lemmatizer"] if lemmatize else []):
        doc = nlp(text)
    toks = []
    for t in doc:
        if not t.is_alpha:
            continue
        if remove_stop and (t.is_stop or t.text in STOP_WORDS):
            continue
        toks.append(t.lemma_ if (lemmatize and t.lemma_ != "-PRON-") else t.text)
    return toks

def type_token_ratio(tokens):
    return (len(set(tokens)) / max(1, len(tokens)))

def jaccard_set(a: set, b: set) -> float:
    if not a and not b: return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def save_heatmap(matrix, labels, title, out_png):
    plt.figure(figsize=(max(6, len(labels)*0.22), max(5, len(labels)*0.22)))
    plt.imshow(matrix, aspect='auto', interpolation='nearest')
    plt.colorbar()
    plt.xticks(range(len(labels)), labels, rotation=90, fontsize=7)
    plt.yticks(range(len(labels)), labels, fontsize=7)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()

def write_csv(df, path):
    df.to_csv(path, index=True)

def discover_docs(root: Path):
    out = []
    for sub in sorted(root.iterdir()):
        if not sub.is_dir(): continue
        name = sub.name
        if name.endswith("_chapters"):
            book = name.replace("_chapters","")
            level = "chapters"
        elif name.endswith("_pages"):
            book = name.replace("_pages","")
            level = "pages"
        else:
            continue
        for md in sorted(sub.glob("*.md")):
            out.append({
                "book": book,
                "level": level,
                "id": md.stem,
                "path": md
            })
    return out

def process_all(items, remove_stop, lemmatize=True, desc="tokenize"):
    nlp = spacy.load("en_core_web_sm")
    rows = []
    for it in tqdm(items, desc=f"{desc} (stopwords_removed={remove_stop})"):
        text = read_markdown_lower(it["path"])
        tokens = tokenize_doc(nlp, text, remove_stop=remove_stop, lemmatize=lemmatize)
        cnt = Counter(tokens)
        rows.append({
            "book": it["book"],
            "level": it["level"],
            "id": it["id"],
            "path": str(it["path"]),
            "n_tokens": len(tokens),
            "n_types": len(cnt),
            "ttr": type_token_ratio(tokens),
            "top_tokens": json.dumps(cnt.most_common(200)),
            "tokens_joined": " ".join(tokens),
            "vocab_set": set(cnt.keys())
        })
    return rows

def build_tfidf_and_cosine(rows, ngram_range=(1,1), min_df=2):
    corpus = [r["tokens_joined"] for r in rows]
    vec = TfidfVectorizer(ngram_range=ngram_range, min_df=min_df)
    X = vec.fit_transform(corpus)  # docs x terms (sparse)
    cos = cosine_similarity(X, dense_output=False)  # sparse/safe
    return vec, X, cos

def build_jaccard_matrix(rows):
    n = len(rows)
    mats = np.zeros((n,n), dtype=np.float32)
    sets = [r["vocab_set"] for r in rows]
    for i in range(n):
        mats[i,i] = 1.0
        si = sets[i]
        for j in range(i+1, n):
            sj = sets[j]
            v = jaccard_set(si, sj)
            mats[i,j] = mats[j,i] = v
    return mats

def build_pearson_corr_from_tfidf(X, max_features=50000):
    # Reduce features to keep memory reasonable
    if X.shape[1] > max_features:
        # keep top-variance features
        variances = np.array((X.power(2)).mean(axis=0) - (X.mean(axis=0).A1**2)).ravel()
        idx = np.argsort(variances)[-max_features:]
        Xr = X[:, idx].toarray()
    else:
        Xr = X.toarray()
    corr = np.corrcoef(Xr)
    corr = np.nan_to_num(corr, nan=0.0, posinf=1.0, neginf=-1.0)
    return corr

def top_pairs_from_matrix(matrix, labels, k=50):
    pairs = []
    n = len(labels)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((labels[i], labels[j], float(matrix[i,j])))
    pairs.sort(key=lambda x: x[2], reverse=True)
    return pairs[:k]

REPORT_HTML = """<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
<title>Book Analysis Report</title>
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; margin: 24px; }
h1, h2, h3 { margin: 12px 0; }
.grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(340px, 1fr)); gap: 16px; }
.card { border: 1px solid #ddd; border-radius: 10px; padding: 12px; }
small { color: #666; }
table { border-collapse: collapse; width: 100%; }
td, th { border: 1px solid #ddd; padding: 6px 8px; }
</style>
</head>
<body>
<h1>Comprehensive Book Analysis</h1>
<p><small>Generated at: {{ generated_at }}</small></p>

<h2>Per-document statistics</h2>
<div class="grid">
{% for sec in stats %}
  <div class="card">
    <h3>{{ sec.title }}</h3>
    <p><a href="{{ sec.csv }}">CSV</a></p>
  </div>
{% endfor %}
</div>

<h2>Matrices & Heatmaps</h2>
<div class="grid">
{% for m in matrices %}
  <div class="card">
    <h3>{{ m.title }}</h3>
    <p><a href="{{ m.csv }}">Matrix CSV</a></p>
    <img src="{{ m.png }}" style="max-width:100%; height:auto;" />
  </div>
{% endfor %}
</div>

<h2>Leaderboards (Top Overlaps)</h2>
<div class="grid">
{% for lb in leaderboards %}
  <div class="card">
    <h3>{{ lb.title }}</h3>
    <p><a href="{{ lb.csv }}">CSV</a></p>
  </div>
{% endfor %}
</div>

</body>
</html>
"""

def run_pipeline():
    all_items = discover_docs(ROOT)
    if not all_items:
        raise SystemExit("No markdown files found under the expected structure.")

    all_items = [it for it in all_items if it["level"] in set(LEVELS)]
    sw_modes = ["no","yes"] if WITH_STOPWORDS == "both" else [WITH_STOPWORDS]

    report_stats, report_mats, report_lbs = [], [], []

    for sw in sw_modes:
        remove_stop = (sw == "no")  # "no stopwords" => remove_stop True
        mode_tag = "nostop" if remove_stop else "withstop"

        rows = process_all(all_items, remove_stop=remove_stop, lemmatize=True,
                           desc="spaCy tokenize")
        # per-doc stats
        stats_df = pd.DataFrame([{
            "book": r["book"],
            "level": r["level"],
            "id": r["id"],
            "path": r["path"],
            "n_tokens": r["n_tokens"],
            "n_types": r["n_types"],
            "ttr": r["ttr"]
        } for r in rows]).sort_values(["book","level","id"])
        stats_csv = OUT / "csv" / f"doc_stats_{mode_tag}.csv"
        write_csv(stats_df, stats_csv)
        report_stats.append({"title": f"Per-doc stats ({mode_tag})",
                             "csv": str(stats_csv.relative_to(OUT))})

        # aggregate top tokens per (book, level)
        agg = defaultdict(Counter)
        for r in rows:
            agg[(r["book"], r["level"])].update(dict(json.loads(r["top_tokens"])))
        agg_rows = []
        for (book, level), cnt in agg.items():
            for tok, c in cnt.most_common(TOPK):
                agg_rows.append({"book":book, "level":level, "token":tok, "count":c})
        top_csv = OUT / "csv" / f"top_tokens_{mode_tag}.csv"
        write_csv(pd.DataFrame(agg_rows), top_csv)
        report_stats.append({"title": f"Top-{TOPK} tokens by book/level ({mode_tag})",
                             "csv": str(top_csv.relative_to(OUT))})

        labels = [f"{r['book']}::{r['level']}::{r['id']}" for r in rows]

        # TF-IDF + Cosine
        vec, X, cos = build_tfidf_and_cosine(rows, ngram_range=NGRAM_RANGE, min_df=MIN_DF)
        cos = cos.A if hasattr(cos, "A") else np.array(cos)

        # Pearson (optional)
        pearson = None
        if DO_PEARSON:
            pearson = build_pearson_corr_from_tfidf(X)

        # Jaccard
        jac = build_jaccard_matrix(rows)

        # Save full matrices
        def save_matrix(mat, name, title):
            df = pd.DataFrame(mat, index=labels, columns=labels)
            csv_path = OUT / "csv" / f"{name}_{mode_tag}.csv"
            write_csv(df, csv_path)
            png_path = OUT / "png" / f"{name}_{mode_tag}.png"
            save_heatmap(mat, labels, title, png_path)
            report_mats.append({"title": f"{title} ({mode_tag})",
                                "csv": str(csv_path.relative_to(OUT)),
                                "png": str(png_path.relative_to(OUT))})

        save_matrix(jac, "jaccard_all", "Jaccard Overlap (All Docs)")
        save_matrix(cos, "cosine_all", "Cosine Similarity (TF-IDF, All Docs)")
        if pearson is not None:
            save_matrix(pearson, "pearson_all", "Pearson Correlation (All Docs)")

        # Intra/Inter per level
        for level in sorted(set(r["level"] for r in rows)):
            idx_level = [i for i,r in enumerate(rows) if r["level"]==level]
            labels_lvl = [labels[i] for i in idx_level]

            def sub(mat):
                return mat[np.ix_(idx_level, idx_level)]

            if "intrabook" in MODES:
                for book in sorted(set(r["book"] for r in rows)):
                    idx = [i for i,r in enumerate(rows) if r["level"]==level and r["book"]==book]
                    if len(idx) < 2: continue
                    labels_sub = [labels[i] for i in idx]
                    for (mat, name, title) in [(jac,"jaccard","Jaccard"),
                                               (cos,"cosine","Cosine")] +                                               ([(pearson,"pearson","Pearson")] if pearson is not None else []):
                        ms = mat[np.ix_(idx, idx)]
                        df = pd.DataFrame(ms, index=labels_sub, columns=labels_sub)
                        csv_path = OUT / "csv" / f"{name}_intrabook_{book}_{level}_{mode_tag}.csv"
                        write_csv(df, csv_path)
                        png_path = OUT / "png" / f"{name}_intrabook_{book}_{level}_{mode_tag}.png"
                        save_heatmap(ms, labels_sub, f"{title} ({book} / {level})", png_path)
                        report_mats.append({"title": f"{title} – Intrabook {book} ({level}, {mode_tag})",
                                            "csv": str(csv_path.relative_to(OUT)),
                                            "png": str(png_path.relative_to(OUT))})
                    # leaderboard
                    pairs = top_pairs_from_matrix(ms, labels_sub, k=50)
                    lb_df = pd.DataFrame(pairs, columns=["doc_a","doc_b","score"])
                    lb_path = OUT / "csv" / f"leaderboard_intrabook_{book}_{level}_{mode_tag}.csv"
                    write_csv(lb_df, lb_path)
                    report_lbs.append({"title": f"Top Overlaps – Intrabook {book} ({level}, {mode_tag})",
                                       "csv": str(lb_path.relative_to(OUT))})

            if "interbook" in MODES:
                for (mat, name, title) in [(jac,"jaccard","Jaccard"),
                                           (cos,"cosine","Cosine")] +                                           ([(pearson,"pearson","Pearson")] if pearson is not None else []):
                    ms = sub(mat)
                    df = pd.DataFrame(ms, index=labels_lvl, columns=labels_lvl)
                    csv_path = OUT / "csv" / f"{name}_interbook_{level}_{mode_tag}.csv"
                    write_csv(df, csv_path)
                    png_path = OUT / "png" / f"{name}_interbook_{level}_{mode_tag}.png"
                    save_heatmap(ms, labels_lvl, f"{title} – Interbook ({level})", png_path)
                    report_mats.append({"title": f"{title} – Interbook ({level}, {mode_tag})",
                                        "csv": str(csv_path.relative_to(OUT)),
                                        "png": str(png_path.relative_to(OUT))})

                pairs = top_pairs_from_matrix(sub(jac), labels_lvl, k=50)
                lb_df = pd.DataFrame(pairs, columns=["doc_a","doc_b","score"])
                lb_path = OUT / "csv" / f"leaderboard_interbook_{level}_{mode_tag}.csv"
                write_csv(lb_df, lb_path)
                report_lbs.append({"title": f"Top Overlaps – Interbook ({level}, {mode_tag})",
                                   "csv": str(lb_path.relative_to(OUT))})

    html = Template(REPORT_HTML).render(
        generated_at=pd.Timestamp.now().isoformat(timespec="seconds"),
        stats=report_stats, matrices=report_mats, leaderboards=report_lbs
    )
    (OUT / "index.html").write_text(html, encoding="utf-8")
    print(f"✅ Done. Open {OUT/'index.html'}")


: 

In [ ]:

# ▶️ Run the full pipeline
run_pipeline()


spaCy tokenize (stopwords_removed=True):   0%|          | 0/3267 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
spaCy tokenize (stopwords_removed=True): 100%|██████████| 3267/3267 [02:19<00:00, 23.46it/s]
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
